In [1]:
pip install opencv-python dlib face_recognition numpy pillow

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install dlib face_recognition opencv-python pillow

Note: you may need to restart the kernel to use updated packages.


In [6]:
# 在檔案開頭確保導入所有必要的模組
import os
import cv2
import dlib
import pickle
import numpy as np
import face_recognition
from PIL import Image, ExifTags
from pathlib import Path
from datetime import datetime

In [31]:
class FaceRecognitionSystem:
    def __init__(self):
        # 創建必要的目錄
        self.train_dir = "training_data"
        self.model_dir = "models"
        self.temp_dir = "temp"
        
        for directory in [self.train_dir, self.model_dir, self.temp_dir]:
            Path(directory).mkdir(exist_ok=True)
            
        # 初始化模型
        self.face_detector = dlib.get_frontal_face_detector()
        # 5點面部標記器，用於人臉對齊
        self.shape_predictor = dlib.shape_predictor("shape_predictor_5_face_landmarks.dat")  
        # 初始化人臉辨識模型
        self.face_recognizer = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")
        
        # OpenCV 的人臉檢測器作為備用
        self.haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        self.profile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_profileface.xml')
        
        # 載入模型（如果存在）
        self.known_face_encodings = []
        self.known_face_names = []
        self.load_model()
    
    def load_model(self):
        """載入先前訓練過的模型"""
        model_path = os.path.join(self.model_dir, "face_encodings.pickle")
        if os.path.exists(model_path):
            with open(model_path, "rb") as f:
                data = pickle.load(f)
                self.known_face_encodings = data["encodings"]
                self.known_face_names = data["names"]
            print(f"模型載入成功，包含 {len(self.known_face_names)} 個人物")
        else:
            print("沒有找到現有模型，需要先訓練")
    
    def add_person(self, person_name, images_folder):
        
        if not os.path.exists(images_folder):
            print(f"錯誤：找不到資料夾 {images_folder}")
            return False
        
        image_paths = [os.path.join(images_folder, f) for f in os.listdir(images_folder) 
                       if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        if not image_paths:
            print(f"錯誤：資料夾 {images_folder} 中沒有圖片檔案")
            return False
        
        face_encodings = []
        successful_images = 0
        
        print(f"開始處理 {person_name} 的 {len(image_paths)} 張圖片...")
        
        for img_path in image_paths:
            try:
                # 使用 PIL 載入圖片（更穩定）
                pil_image = Image.open(img_path)
                # 轉換為 RGB（移除 alpha 通道如果有的話）
                if pil_image.mode != 'RGB':
                    pil_image = pil_image.convert('RGB')
                # 轉換為 numpy 數組給 face_recognition 使用
                img = np.array(pil_image)
                
                # 檢測人臉 - 使用 face_recognition 庫（基於 dlib）
                face_locations = face_recognition.face_locations(img, model="hog")
                
                # 如果找不到人臉，嘗試使用 OpenCV 的正面和側面檢測器
                if not face_locations:
                    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
                    # 先檢測正面
                    faces = self.haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
                    if len(faces) == 0:
                        # 再檢測側面
                        faces = self.profile_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
                    
                    if len(faces) > 0:
                        # 轉換 OpenCV 坐標到 face_recognition 格式 (top, right, bottom, left)
                        x, y, w, h = faces[0]
                        face_locations = [(y, x+w, y+h, x)]
                
                # 如果還是找不到人臉
                if not face_locations:
                    print(f"警告：無法在 {img_path} 中檢測到人臉")
                    continue
                
                # 計算人臉特徵編碼
                encodings = face_recognition.face_encodings(img, face_locations)
                
                if encodings:
                    face_encodings.extend(encodings)
                    successful_images += 1
                    print(f"成功處理: {img_path}")
                else:
                    print(f"警告：無法在 {img_path} 中創建有效的人臉編碼")
            
            except Exception as e:
                print(f"處理 {img_path} 時發生錯誤: {e}")
        
        print(f"成功處理 {successful_images}/{len(image_paths)} 張圖片")
        
        # 如果有成功處理的圖片，將他們加入到模型中
        if face_encodings:
            self.known_face_encodings.extend(face_encodings)
            self.known_face_names.extend([person_name] * len(face_encodings))
            
            # 儲存模型
            self.save_model()
            return True
        else:
            print(f"錯誤：無法為 {person_name} 創建任何有效的人臉編碼")
            return False
    
    def save_model(self):
        """保存模型到檔案"""
        data = {
            "encodings": self.known_face_encodings,
            "names": self.known_face_names
        }
        
        with open(os.path.join(self.model_dir, "face_encodings.pickle"), "wb") as f:
            pickle.dump(data, f)
        
        print(f"模型已保存，包含 {len(self.known_face_names)} 個人臉")
    
    def recognize_face(self, image_path, tolerance=0.6):
    
        if not self.known_face_encodings:
            print("錯誤：模型尚未訓練，請先添加人物")
            return None
    
        try:
        # 載入圖片並處理 EXIF 方向
            pil_image = Image.open(image_path)
        
        # 檢查和修正圖片方向
            try:
                exif = pil_image._getexif()
                if exif:
                    orientation_key = 274  # EXIF 中的 Orientation 標籤
                    if orientation_key in exif:
                        orientation = exif[orientation_key]
                        # 根據方向標籤旋轉圖片
                        if orientation == 2:
                            pil_image = pil_image.transpose(Image.FLIP_LEFT_RIGHT)
                        elif orientation == 3:
                            pil_image = pil_image.rotate(180)
                        elif orientation == 4:
                            pil_image = pil_image.rotate(180).transpose(Image.FLIP_LEFT_RIGHT)
                        elif orientation == 5:
                            pil_image = pil_image.rotate(-90).transpose(Image.FLIP_LEFT_RIGHT)
                        elif orientation == 6:
                            pil_image = pil_image.rotate(-90)
                        elif orientation == 7:
                            pil_image = pil_image.rotate(90).transpose(Image.FLIP_LEFT_RIGHT)
                        elif orientation == 8:
                            pil_image = pil_image.rotate(90)
                        print(f"已修正圖片方向 (EXIF Orientation: {orientation})")
            except (AttributeError, KeyError, IndexError) as e:
                # 有些圖片可能沒有 EXIF 資料
                pass
            
            if pil_image.mode != 'RGB':
                pil_image = pil_image.convert('RGB')
        
            # 保存原始尺寸以便最後輸出時保持相同
            original_size = pil_image.size  # (width, height)
            original_orientation = 'portrait' if original_size[1] > original_size[0] else 'landscape'
        
            img = np.array(pil_image)
            # 創建可以繪製的副本
            output_img = img.copy()
        
            # 檢測所有人臉
            face_locations = face_recognition.face_locations(img, model="hog")
        
            # 如果找不到人臉，使用 OpenCV 嘗試檢測正面和側面
            if not face_locations:
                gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
                # 檢測正面
                frontal_faces = self.haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
                for (x, y, w, h) in frontal_faces:
                    face_locations.append((y, x+w, y+h, x))
            
                # 檢測側面
                profile_faces = self.profile_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
                for (x, y, w, h) in profile_faces:
                    # 檢查是否與已檢測的面孔重疊
                    is_new = True
                    for (top, right, bottom, left) in face_locations:
                        # 如果中心點落在已有的人臉框內，則不添加
                        center_x, center_y = x + w//2, y + h//2
                        if left <= center_x <= right and top <= center_y <= bottom:
                            is_new = False
                            break
                    if is_new:
                        face_locations.append((y, x+w, y+h, x))
        
            if not face_locations:
                print(f"在圖片中未檢測到任何人臉")
                return output_img
        
            # 計算檢測到的人臉的特徵值
            face_encodings = face_recognition.face_encodings(img, face_locations)
        
            # 顯示處理進度
            print(f"檢測到 {len(face_locations)} 個人臉")
        
            # 辨識每個人臉
            for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                # 與所有已知人臉比對
                matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding, tolerance=tolerance)
                name = "未知"
            
                # 如果有匹配項，使用距離最近的那個
                if True in matches:
                    face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
                    best_match_index = np.argmin(face_distances)
                    confidence = 1 - face_distances[best_match_index]
                    if matches[best_match_index]:
                        name = f"{self.known_face_names[best_match_index]} ({confidence:.2f})"
            
                # 繪製人臉框和名字
                cv2.rectangle(output_img, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.rectangle(output_img, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
                cv2.putText(output_img, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 255), 1)
        
            return output_img
    
        except Exception as e:
            print(f"識別過程中發生錯誤: {e}")
            return None
    
    def recognize_from_webcam(self, camera_id=0, tolerance=0.6):
        """從網路攝影機識別人臉"""
        if not self.known_face_encodings:
            print("錯誤：模型尚未訓練，請先添加人物")
            return
        
        # 開啟攝影機
        cap = cv2.VideoCapture(camera_id)
        
        if not cap.isOpened():
            print("錯誤：無法開啟攝影機")
            return
        
        print("開始從攝影機識別人臉。按空白鍵結束。")
        
        process_this_frame = True
        
        while True:
            # 讀取一幀
            ret, frame = cap.read()
            
            if not ret:
                print("無法從攝影機獲取畫面")
                break
            
            # 只處理每隔一幀以提高效率
            if process_this_frame:
                # 縮小畫面以加快處理速度
                small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
                rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
                
                # 檢測人臉
                face_locations = face_recognition.face_locations(rgb_small_frame, model="hog")
                
                # 如果找不到人臉，使用 OpenCV 嘗試檢測正面和側面
                if not face_locations:
                    gray = cv2.cvtColor(small_frame, cv2.COLOR_BGR2GRAY)
                    
                    # 檢測正面
                    frontal_faces = self.haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
                    for (x, y, w, h) in frontal_faces:
                        face_locations.append((y, x+w, y+h, x))
                    
                    # 檢測側面
                    profile_faces = self.profile_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
                    for (x, y, w, h) in profile_faces:
                        # 檢查是否與已檢測的面孔重疊
                        is_new = True
                        for (top, right, bottom, left) in face_locations:
                            center_x, center_y = x + w//2, y + h//2
                            if left <= center_x <= right and top <= center_y <= bottom:
                                is_new = False
                                break
                        if is_new:
                            face_locations.append((y, x+w, y+h, x))
                
                # 如果有檢測到人臉，進行識別
                face_names = []
                if face_locations:
                    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
                    
                    for face_encoding in face_encodings:
                        # 與已知人臉比對
                        matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding, tolerance=tolerance)
                        name = "未知"
                        
                        if True in matches:
                            face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
                            best_match_index = np.argmin(face_distances)
                            confidence = 1 - face_distances[best_match_index]
                            if matches[best_match_index]:
                                name = f"{self.known_face_names[best_match_index]} ({confidence:.2f})"
                        
                        face_names.append(name)
            
            process_this_frame = not process_this_frame
            
            # 顯示結果
            for (top, right, bottom, left), name in zip(face_locations, face_names):
                # 因為偵測是在縮小的畫面上進行的，所以要調整座標
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4
                
                # 畫人臉框
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                
                # 畫名字背景
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
                cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 255), 1)
            
            # 顯示結果
            cv2.imshow('人臉辨識', frame)
            
            if cv2.waitKey(1) & 0xFF == ord(' '):
                break
        
        # 釋放資源
        cap.release()
        cv2.destroyAllWindows()
    def recognize_from_video(self, video_path, output_path=None, tolerance=0.6, process_every_n_frames=5):

        if not self.known_face_encodings:
            print("錯誤：模型尚未訓練，請先添加人物")
            return None
    
        # 開啟影片
        cap = cv2.VideoCapture(video_path)
    
        if not cap.isOpened():
            print(f"錯誤：無法開啟影片檔案 {video_path}")
            return None
    
        # 獲取影片屬性
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
        print(f"影片資訊：")
        print(f"- 解析度：{width}x{height}")
        print(f"- FPS：{fps}")
        print(f"- 總幀數：{total_frames}")
        print(f"- 時長：{total_frames/fps:.2f} 秒")
    
        # 如果需要儲存輸出影片
        out = None
        if output_path:
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
        # 統計結果
        recognition_stats = {}
        frame_count = 0
        last_face_locations = []
        last_face_names = []
    
        print(f"開始處理影片...")
    
        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
            
                # 顯示進度
                if frame_count % 30 == 0:
                    progress = (frame_count / total_frames) * 100
                    print(f"處理進度：{progress:.1f}% ({frame_count}/{total_frames} 幀)")
            
                # 只處理每第 n 幀以提高效能
                if frame_count % process_every_n_frames == 0:
                    # 縮小畫面以加快處理速度
                    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
                    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
                
                    # 檢測人臉
                    face_locations = face_recognition.face_locations(rgb_small_frame, model="hog")
                
                    # 如果找不到人臉，使用 OpenCV 嘗試檢測
                    if not face_locations:
                        gray = cv2.cvtColor(small_frame, cv2.COLOR_BGR2GRAY)
                        faces = self.haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
                        profile_faces = self.profile_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
                    
                        # 合併正面和側面檢測結果
                        all_faces = list(faces) + list(profile_faces)
                        for (x, y, w, h) in all_faces:
                            face_locations.append((y, x+w, y+h, x))
                
                    # 如果有檢測到人臉，進行識別
                    face_names = []
                    if face_locations:
                        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
                    
                        for face_encoding in face_encodings:
                            # 與已知人臉比對
                            matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding, tolerance=tolerance)
                            name = "未知"
                        
                            if True in matches:
                                face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
                                best_match_index = np.argmin(face_distances)
                                confidence = 1 - face_distances[best_match_index]
                                if matches[best_match_index]:
                                    name = self.known_face_names[best_match_index]
                                    # 更新統計
                                    if name not in recognition_stats:
                                        recognition_stats[name] = {
                                            'count': 0,
                                            'first_seen_frame': frame_count,
                                            'last_seen_frame': frame_count,
                                            'confidences': []
                                        }
                                    recognition_stats[name]['count'] += 1
                                    recognition_stats[name]['last_seen_frame'] = frame_count
                                    recognition_stats[name]['confidences'].append(confidence)
                        
                            face_names.append(name)
                
                    # 更新暫存的人臉位置和名稱
                    last_face_locations = face_locations
                    last_face_names = face_names
            
                # 在畫面上繪製結果（使用上一次的檢測結果）
                for (top, right, bottom, left), name in zip(last_face_locations, last_face_names):
                    # 因為偵測是在縮小的畫面上進行的，所以要調整座標
                    top *= 4
                    right *= 4
                    bottom *= 4
                    left *= 4
                
                    # 畫人臉框
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                
                    # 畫名字背景
                    cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
                    cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 255), 1)
            
                # 在畫面左上角顯示幀數和時間
                time_text = f"幀：{frame_count} | 時間：{frame_count/fps:.2f}s"
                cv2.putText(frame, time_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            
                # 如果需要儲存輸出影片
                if out:
                    out.write(frame)
            
                # 顯示結果（可選）
                if frame_count % process_every_n_frames == 0:  # 減少顯示頻率以提高效能
                    cv2.imshow('影片人臉識別', frame)
                    if cv2.waitKey(1) & 0xFF == ord(' '):
                        print("使用者中斷處理")
                        break
            
                frame_count += 1
    
        finally:
            # 釋放資源
            cap.release()
            if out:
                out.release()
            cv2.destroyAllWindows()
    
        # 計算識別結果摘要
        print("\n識別結果摘要：")
        for name, stats in recognition_stats.items():
            avg_confidence = sum(stats['confidences']) / len(stats['confidences'])
            first_time = stats['first_seen_frame'] / fps
            last_time = stats['last_seen_frame'] / fps
            duration = last_time - first_time
        
            print(f"\n{name}:")
            print(f"  - 出現次數：{stats['count']} 次")
            print(f"  - 首次出現：{first_time:.2f} 秒")
            print(f"  - 最後出現：{last_time:.2f} 秒")
            print(f"  - 出現總時長：{duration:.2f} 秒")
            print(f"  - 平均信心度：{avg_confidence:.2f}")
    
        return recognition_stats

# 使用示例
if __name__ == "__main__":
    # 初始化系統
    face_system = FaceRecognitionSystem()
    
    # 選擇操作模式
    print("請選擇操作模式：")
    print("1. 添加新人物")
    print("2. 從圖片識別")
    print("3. 從攝影機識別")
    print("4. 從影片識別")
    
    choice = input("請輸入選項(1-4): ")
    
    if choice == "1":
        person_name = input("請輸入人物名稱: ")
        images_folder = input("請輸入包含該人物照片的資料夾路徑: ")
        face_system.add_person(person_name, images_folder)
    
    elif choice == "2":
        image_path = input("請輸入要識別的圖片路徑: ")
        result_img = face_system.recognize_face(image_path)
        
        if result_img is not None:
            # 使用 PIL 來保存，這樣可以更好地保持方向
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            result_path = os.path.join("temp", f"result_{timestamp}.jpg")
            
            # 將 numpy 數組轉回 PIL 圖像
            result_pil = Image.fromarray(result_img)
            
            # 保存圖片
            result_pil.save(result_path, quality=95)
            print(f"結果已保存至 {result_path}")
    
    elif choice == "3":
        face_system.recognize_from_webcam()
    
    elif choice == "4":
        video_path = input("請輸入影片檔案路徑: ")
        save_output = input("是否要儲存處理後的影片？(y/n): ").lower() == 'y'
        
        output_path = None
        if save_output:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_path = os.path.join("temp", f"result_video_{timestamp}.avi")
        
        stats = face_system.recognize_from_video(video_path, output_path=output_path)
        
        if stats:
            print("\n影片處理完成！")
            if output_path:
                print(f"輸出影片已儲存至：{output_path}")
    
    else:
        print("無效的選項")

模型載入成功，包含 69 個人物
請選擇操作模式：
1. 添加新人物
2. 從圖片識別
3. 從攝影機識別
4. 從影片識別
開始從攝影機識別人臉。按空白鍵結束。
